In [1]:
# !git clone https://github.com/alexeygrigorev/mlbookcamp-code.git

# %cd mlbookcamp-code/course-zoomcamp/cohorts/2022/05-deployment/homework/
# !docker build -t zoomcamp-model:v001 .

In [2]:
# Run in other jupyter session/external terminal
# %cd mlbookcamp-code/course-zoomcamp/cohorts/2022/05-deployment/homework/
# !docker run -it --rm -p 9696:9696 zoomcamp-model:v001

#### Q1. What is probability of getting a credit card?

%cd mlbookcamp-code/course-zoomcamp/cohorts/2022/05-deployment/homework/
!python q6_test.py

#### 0.769

Install kubectl and kind

#### Q2. What's the version of kind that you have?

In [3]:
!kind --version

kind version 0.17.0


#### 0.17.0

#### Q3. What's the smallest deployable computing unit that we can create and manage in Kubernetes (kind in our case)?

#### Pod

#### Q4. What's the Type of the service that is already running there?

#### ClusterIP

To be able to use the docker image we previously created (zoomcamp-model:v001), we need to register it with kind.

#### Q5. What's the command we need to run for that?

#### kind load docker-image zoomcamp-model:v001

Create a deployment config (e.g. deployment.yaml)

#### Q6. What is the value for <Port>?

#### 9696

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

#### Q7. create a service for this deployment (service.yaml)

Fill it in. What do we need to write instead of <???>?

#### credit-score

Apply this config file.

#### Testing the service

We can test our service locally by forwarding the port 9696 on our computer to the port 80 on the service:

kubectl port-forward service/<Service name> 9696:80

Run q6_test.py (from the homework 5) once again to verify that everything is working. You should get the same result as in Question 1.

#### Autoscaling

Now we're going to use a HorizontalPodAutoscaler (HPA for short) that automatically updates a workload resource (such as our deployment), with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3

You can check the current status of the new HPA by running:

kubectl get hpa

The output should be similar to the next:

NAME              REFERENCE                TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   1%/20%    1         3         1          27s

TARGET column shows the average CPU consumption across all the Pods controlled by the corresponding deployment. Current CPU consumption is about 0% as there are no clients sending requests to the server.

    Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release from the components.yaml manifest:

    kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

#### Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing q6_test.py script by putting the operator that sends the request to the credit-card service into a loop.

while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)

Now you can run this script.

Run kubectl get hpa credit-card-hpa --watch command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas.
#### Q8. What was the maximum amount of the replicas during this test?

Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled in any way it may happen that the final number of replicas will differ from initial.